<a href="https://colab.research.google.com/github/sthalles/SimCLR/blob/simclr-refactor/feature_eval/mini_batch_logistic_regression_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import sys
import numpy as np
import os
import yaml
import matplotlib.pyplot as plt
import torchvision
import pandas as pd

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def get_file_id_by_model(folder_name):
  file_id = {'resnet18_100-epochs_stl10': '14_nH2FkyKbt61cieQDiSbBVNP8-gtwgF',
             'resnet18_100-epochs_cifar10': '1lc2aoVtrAetGn0PnTkOyFzPCIucOJq7C',
             'resnet50_50-epochs_stl10': '1ByTKAUsdm_X7tLcii6oAEl5qFRqRMZSu'}
  return file_id.get(folder_name, "Model not found.")

In [ ]:
folder_name = 'resnet50_50-epochs_stl10'
file_id = get_file_id_by_model(folder_name)
print(folder_name, file_id)

resnet50_50-epochs_stl10 1ByTKAUsdm_X7tLcii6oAEl5qFRqRMZSu


In [ ]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
from torch.nn.functional import interpolate

In [ ]:
dataset_list = ['stl10','cifar10']
dataset = dataset_list[0]
models = ['rgb3000im_50ep','deptheuclidean3000im_50ep']
model = models[0]
epochs = 100

if model == 'rgb3000im_50ep':
  model_weights_dest = '/content/drive/MyDrive/trained_models/rgb/epoch=49-step=4800.ckpt'
elif model== 'deptheuclidean3000im_50ep':
  model_weights_dest = '/content/drive/MyDrive/trained_models/depth_euclidean/epoch=49-step=4800.ckpt'

save_file_name = f'{model}_img_resize_linear_regression_eval_{dataset}_Finetune{epochs}ep'

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

Using device: cuda


In [ ]:
def get_stl10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.STL10('./data', split='train', download=download,
                                  transform=transforms.Compose([
                                      transforms.Resize(32),
                                      transforms.ToTensor()]))
                                      
  #train_dataset = interpolate(train_dataset,size=(32,32),scale_factor=(2,2))

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=0, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.STL10('./data', split='test', download=download,
                                  transform=transforms.Compose([
                                       transforms.Resize(32),
                                      transforms.ToTensor()]))
  
  #test_dataset = interpolate(test_dataset,size=(32,32),scale_factor=(2,2))

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=10, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

def get_cifar10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.CIFAR10('./data', train=True, download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=0, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.CIFAR10('./data', train=False, download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=10, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

In [ ]:
model = torchvision.models.resnet50(pretrained=False, num_classes=10).to(device)

In [ ]:
!pip install lightning-bolts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
MODEL_PATH = model_weights_dest
checkpoint = torch.load(MODEL_PATH, map_location=device)
state_dict = checkpoint['state_dict']

for k in list(state_dict.keys()):

  if k.startswith('backbone.'):
    if k.startswith('backbone') and not k.startswith('backbone.fc'):
      # remove prefix
      state_dict[k[len("backbone."):]] = state_dict[k]
  del state_dict[k]

In [ ]:
if dataset == 'cifar10':
  train_loader, test_loader = get_cifar10_data_loaders(download=True)
elif dataset == 'stl10':
  train_loader, test_loader = get_stl10_data_loaders(download=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# freeze all layers but the last fc
for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        param.requires_grad = False

parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
assert len(parameters) == 2  # fc.weight, fc.bias

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=0.0008)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [ ]:
data ={'Epoch':[],'Top1 Train Accuracy':[], 'Top1 Test Accuracy':[], 'Top5 Test Accuracy':[]}
df_testdata = pd.DataFrame(data)

for epoch in range(epochs):
  top1_train_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(train_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
    loss = criterion(logits, y_batch)
    top1 = accuracy(logits, y_batch, topk=(1,))
    top1_train_accuracy += top1[0]

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  top1_train_accuracy /= (counter + 1)
  top1_accuracy = 0
  top5_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(test_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
  
    top1, top5 = accuracy(logits, y_batch, topk=(1,5))
    top1_accuracy += top1[0]
    top5_accuracy += top5[0]
  
  top1_accuracy /= (counter + 1)
  top5_accuracy /= (counter + 1)
  print(f"Epoch {epoch}\tTop1 Train accuracy {top1_train_accuracy.item()}\tTop1 Test accuracy: {top1_accuracy.item()}\tTop5 test acc: {top5_accuracy.item()}")
  epoch_data = {'Epoch':epoch,'Top1 Train Accuracy':top1_train_accuracy.item(), 'Top1 Test Accuracy':top1_accuracy.item(), 'Top5 Test Accuracy':top5_accuracy.item()}
  df_testdata = df_testdata.append(pd.Series(data=epoch_data,name='x'), ignore_index=True)
df_testdata.to_csv(f'{save_file_name}.csv')
    

Epoch 0	Top1 Train accuracy 10.76056957244873	Top1 Test accuracy: 12.021484375	Top5 test acc: 53.9501953125
Epoch 1	Top1 Train accuracy 14.611673355102539	Top1 Test accuracy: 12.529296875	Top5 test acc: 55.72021484375
Epoch 2	Top1 Train accuracy 17.420726776123047	Top1 Test accuracy: 13.0322265625	Top5 test acc: 56.58203125
Epoch 3	Top1 Train accuracy 20.38832664489746	Top1 Test accuracy: 13.30322265625	Top5 test acc: 57.52685546875
Epoch 4	Top1 Train accuracy 23.327207565307617	Top1 Test accuracy: 13.564453125	Top5 test acc: 58.32275390625
Epoch 5	Top1 Train accuracy 26.81296157836914	Top1 Test accuracy: 13.92578125	Top5 test acc: 58.97705078125
Epoch 6	Top1 Train accuracy 29.858686447143555	Top1 Test accuracy: 13.97705078125	Top5 test acc: 59.5703125
Epoch 7	Top1 Train accuracy 32.44025802612305	Top1 Test accuracy: 14.09423828125	Top5 test acc: 60.2880859375
Epoch 8	Top1 Train accuracy 34.78745651245117	Top1 Test accuracy: 14.43603515625	Top5 test acc: 60.7373046875
Epoch 9	Top1 Trai